# 🍰 [Day 5](https://adventofcode.com/2020/day/5)

In [1]:
def parse(lines):
    return [[1 if c in ['F', 'L'] else 0 for c in line] for line in lines]

def binary_search(encoding, start, end):
    """Recursive function to get row or column from binary encoding"""
    if len(encoding):
        mid = (start + end) // 2
        return binary_search(encoding[1:], 
                             start if encoding[0] else mid + 1,
                             mid if encoding[0] else end)
    else:
        assert start == end
        return start

def get_id(encoding):
    """Get ID"""
    row = binary_search(encoding[:7], 0, 127)
    col = binary_search(encoding[7:10], 0, 7)
    return row * 8 + col
    
def get_ids(lines):
    return [get_id(encoding) for encoding in parse(lines)]

def find_my_seat(lines):
    """Because there's only 8 columns, the ID is unique per seat"""
    seats = {row * 8 + col: True for row in range(128) for col in range(8)}
    for encoding in parse(lines):
        del seats[get_id(encoding)]
    keys = []
    # Find the only keys whose neighbors are not in the list
    for k in seats:
        if not (seats.get(k - 1, False) or seats.get(k + 1, False)):
            keys.append(k)
    assert len(keys) == 1
    return keys[0], keys[0] // 8, chr(ord('A') + keys[0] % 8)

In [2]:
with open('inputs/day05.txt', 'r') as f:
    inputs = f.read().splitlines()
    
print(f"The highest seat IDs in the boarding passes in {max(get_ids(inputs))}")
print("I found my seat right in time ! it's {1}{2} (ID = {0})".format(*find_my_seat(inputs)))

The highest seat IDs in the boarding passes in 908
I found my seat right in time ! it's 77D (ID = 619)


In [3]:
from PIL import Image
import numpy as np
import pixelhouse as ph

def boarding(lines, save=True):
    # Save sequence of images
    imgs = []
    
    ## Init canvas
    w, h = 1400, 150
    extent_w = 64
    extent_h = extent_w * h / w
    pal = ph.palette(3)
    canvas = ph.Canvas(w, h, shift=0, extent=extent_w, bg=pal[0])
    
    ## Build the plane structure
    offset_edges = 1
    offset_seats = 0.5
    step_h = (extent_h - offset_edges - offset_seats) * 2 / 8
    step_w = (extent_w - offset_edges - 2 * offset_seats) * 2 / 128
    col_offsets = [- extent_h + offset_edges + (2 * offset_seats if col > 3 else 0) for col in range(8)]
    row_offsets = [- extent_w + offset_edges + 2 * offset_seats * row // 32 for row in range(128)]
    for row in range(128):
        for col in range(8):
            canvas += ph.rectangle(
                row_offsets[row] + step_w * row,
                col_offsets[col] + col * step_h, 
                row_offsets[row] + step_w * (row + 1) - offset_seats,
                col_offsets[col] + (col + 1) * step_h - offset_seats,
                color='white', thickness=0.05)
    ## Add rows and columns names
    for row in range(128):
        canvas += ph.text(str(row), x=row_offsets[row] + step_w * row + (step_w - offset_seats) / 2, 
                          y=extent_h - 0.5, font_size=0.8)
    for col in range(8):
        canvas += ph.text(chr(ord('A') + col), 
                          x=-extent_w + 0.4, 
                          y=col_offsets[col] + step_h * col + (step_h - offset_seats) / 2, font_size=0.8)
    imgs.append(Image.fromarray(np.array(canvas.img)))
    
    ## Board passengers one by one
    seats = {row * 8 + col: True for row in range(128) for col in range(8)}
    for encoding in parse(lines):
        row, col = divmod(get_id(encoding), 8)
        del seats[row * 8 + col]
        canvas += ph.rectangle(
            row_offsets[row] + step_w * row,
            col_offsets[col] + col * step_h, 
            row_offsets[row] + step_w * (row + 1) - offset_seats,
            col_offsets[col] + (col + 1) * step_h - offset_seats,
            color=pal[-2], thickness=-1)
        imgs.append(Image.fromarray(np.array(canvas.img)))
        
    # Find my seat
    keys = []
    for k in seats:
        if not (seats.get(k - 1, False) or seats.get(k + 1, False)):
            keys.append(k)
    assert len(keys) == 1
    
    # Plot my seat
    row, col = divmod(keys[0], 8)
    canvas += ph.rectangle(
        row_offsets[row] + step_w * row,
        col_offsets[col] + col * step_h,
        row_offsets[row] + step_w * (row + 1) - offset_seats,
        col_offsets[col] + (col + 1) * step_h - offset_seats,
        color=pal[2], thickness=-1)
    canvas += ph.text("Boarding complete !", y=1.5, font_size=6)
    imgs.extend([Image.fromarray(np.array(canvas.img))] * 70)
    
    # Save
    canvas.save("viz/day05.png")
    imgs[0].save(fp="viz/day05.gif", format='GIF', append_images=imgs[1:],
                 save_all=True, duration=20, loop=0)

In [4]:
%%time
boarding(inputs)

CPU times: user 8.9 s, sys: 1.18 s, total: 10.1 s
Wall time: 10.2 s


The plane is boarding, please make way to your seat...

![Boarding animation](viz/day05.gif)